In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.6 MB/s eta 0:00:00


In [5]:
import numpy as np
import pandas as pd
import random
import re
import nltk
from nltk import sent_tokenize
nltk.download("punkt")
import openai
from kaggle_secrets import UserSecretsClient

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
user_secrets = UserSecretsClient()
openai_api_key = user_secrets.get_secret("openai_api_key")
#openai_org = user_secrets.get_secret("openai_org")


In [30]:
d_2010 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2010.csv')
d_2011 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2011.csv')
d_2012 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2012.csv')
d_2013 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2013.csv')
d_2014 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2014.csv')
d_2015 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2015.csv')
d_2016 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2016.csv')
d_2017 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2017.csv')
d_2018 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2018.csv')
d_2019 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2019.csv')
d_2020 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2020.csv')
d_2021 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2021.csv')
d_2022 = pd.read_csv('/kaggle/input/elon-musk-tweets-2010-2021/2022.csv')

In [31]:
d_2011.columns

Index(['Unnamed: 0', 'id', 'conversation_id', 'created_at', 'date', 'timezone',
       'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id',
       'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls',
       'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies',
       'nretweets', 'quote_url', 'search', 'near', 'geo', 'source',
       'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date',
       'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [32]:
d_2021.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [46]:
common_columns = list(set(d_2010.columns) & set(d_2011.columns) & set(d_2012.columns) & set(d_2013.columns) & set(d_2014.columns)
                      & set(d_2015.columns) & set(d_2016.columns) & set(d_2017.columns) & set(d_2018.columns) & set(d_2019.columns)
                      & set(d_2020.columns) & set(d_2021.columns) & set(d_2022.columns))
d_2010_common = d_2010[common_columns]
d_2011_common = d_2011[common_columns]
d_2012_common = d_2012[common_columns]
d_2013_common = d_2013[common_columns]
d_2014_common = d_2014[common_columns]
d_2015_common = d_2015[common_columns]
d_2016_common = d_2016[common_columns]
d_2017_common = d_2017[common_columns]
d_2018_common = d_2018[common_columns]
d_2019_common = d_2019[common_columns]
d_2020_common = d_2020[common_columns]
d_2021_common = d_2021[common_columns]
d_2022_common = d_2022[common_columns]
dataset = pd.concat([d_2010_common, d_2011_common, d_2012_common, d_2013_common, d_2014_common, d_2015_common, d_2016_common, 
                     d_2017_common, d_2018_common, d_2019_common, d_2020_common, d_2021_common, d_2022_common])
dataset.reset_index(drop=True, inplace=True)
dataset.to_csv('dataset.csv', index=False)

In [47]:
data = pd.read_csv('/kaggle/working/dataset.csv')

In [48]:
data.shape

(34878, 31)

In [49]:
data.columns

Index(['quote_url', 'link', 'video', 'thumbnail', 'user_rt_id', 'cashtags',
       'language', 'trans_dest', 'tweet', 'id', 'photos', 'geo', 'timezone',
       'user_rt', 'reply_to', 'retweet', 'near', 'username', 'translate',
       'retweet_date', 'hashtags', 'name', 'trans_src', 'place', 'date',
       'urls', 'retweet_id', 'created_at', 'user_id', 'conversation_id',
       'source'],
      dtype='object')

In [50]:
display(data[34875:])

,quote_url,link,video,thumbnail,user_rt_id,cashtags,language,trans_dest,tweet,id,...,name,trans_src,place,date,urls,retweet_id,created_at,user_id,conversation_id,source
34875,https://twitter.com/Tesla/status/1477691175897...,https://twitter.com/elonmusk/status/1477700424...,0,NaN,NaN,[],en,NaN,Great work by Tesla team worldwide!,1477700424933253123,...,Elon Musk,NaN,NaN,2022-01-02,[],NaN,2022-01-02 21:56:50 Arabian Standard Time,44196397,1477700424933253123,NaN
34876,NaN,https://twitter.com/elonmusk/status/1477096955...,0,NaN,NaN,[],und,NaN,@BLKMDL3 @Tesla 🔥,1477096955658579974,...,Elon Musk,NaN,NaN,2022-01-01,[],NaN,2022-01-01 05:58:52 Arabian Standard Time,44196397,1476958174439567360,NaN
34877,NaN,https://twitter.com/elonmusk/status/1477080438...,0,NaN,NaN,[],und,NaN,@MiFSDBetaTester @WholeMarsBlog 🤣,1477080438049058818,...,Elon Musk,NaN,NaN,2022-01-01,[],NaN,2022-01-01 04:53:13 Arabian Standard Time,44196397,1476768985509212162,NaN


In [51]:
data.drop(columns=['quote_url', 'link', 'video', 'thumbnail', 'user_rt_id', 'cashtags', 'trans_dest', 'user_rt', 'reply_to',
                   'retweet', 'near', 'translate', 'retweet_date', 'trans_src', 'retweet_id', 'conversation_id', 'source'], inplace=True)

In [52]:
data.shape

(34878, 14)

In [53]:
display(data[34875:])

,language,tweet,id,photos,geo,timezone,username,hashtags,name,place,date,urls,created_at,user_id
34875,en,Great work by Tesla team worldwide!,1477700424933253123,[],NaN,400,elonmusk,[],Elon Musk,NaN,2022-01-02,[],2022-01-02 21:56:50 Arabian Standard Time,44196397
34876,und,@BLKMDL3 @Tesla 🔥,1477096955658579974,[],NaN,400,elonmusk,[],Elon Musk,NaN,2022-01-01,[],2022-01-01 05:58:52 Arabian Standard Time,44196397
34877,und,@MiFSDBetaTester @WholeMarsBlog 🤣,1477080438049058818,[],NaN,400,elonmusk,[],Elon Musk,NaN,2022-01-01,[],2022-01-01 04:53:13 Arabian Standard Time,44196397


In [54]:
data['texts']=data['date'].apply(lambda x:x[0:7])+' '+data['tweet']
TEXTS=data['texts'].tolist()
print(TEXTS[0:3])
print(len(TEXTS))

['2010-06 Please ignore prior tweets, as that was someone pretending to be me :)  This is actually me.', '2011-12 @TheOnion So true :)', '2011-12 If you ever wanted to know the *real* truth about the moon landings ...(best Onion article ever)  http://t.co/pgNEJsjI']
34878


In [55]:
def get_transcript_as_sentences(paragraphs):
    num=0
    texts=[]
    for p in paragraphs[num:]:
        text = p.strip()
        texts+=[text]
    transcript = ' '.join([string for string in texts])
    transcript = re.sub(r"http\S+", "", transcript)
    transcript = re.sub(r"\S*@\S*\s?", "", transcript)
    sentences = sent_tokenize(transcript)    
    return sentences

In [56]:
sentence = get_transcript_as_sentences(TEXTS)
sentences = random.sample(sentence, 100)
print(len(sentence),len(sentences))

30469 100


In [57]:
def how_many_tokens(text):
    num_chars = len(text)
    num_tokens = round(num_chars/4)
    return num_tokens    
transcript = " ".join(sentences)
print("Transcript size: {} Tokens".format(how_many_tokens(transcript)))

Transcript size: 2141 Tokens


In [75]:
chunk = transcript
print("Chunk: {} Tokens".format(how_many_tokens(chunk)))

Chunk: 2141 Tokens


In [76]:
def get_summaries_from_transcript(transcript_chunk, prompt, is_test=True):
    if is_test:
        return ""    
    openai.api_key = openai_api_key
    transcript_with_prompt = transcript_chunk + "\n" + prompt
    output = openai.Completion.create(engine="text-davinci-003",
                                      prompt=transcript_with_prompt,                                                                                 
                                      temperature=1.0,
                                      max_tokens=1000,
                                      top_p=1,
                                      frequency_penalty=0,
                                      presence_penalty=0)
    
    return output["choices"][0]["text"].strip()

In [78]:
response = get_summaries_from_transcript(chunk[0:100], prompt, is_test=False)

In [82]:
prompt = "What did Elon Musk tweet the most about?"
response = get_summaries_from_transcript(chunk[0:100], prompt, is_test=False)
print(response)

Elon Musk has a wide range of topics that he tweets about, including SpaceX and Tesla announcements, technology advancements, environmental topics, and his opinions on various topics.


In [83]:
prompt = "What was Elon Musk's most funniest tweet?"
response = get_summaries_from_transcript(chunk[0:100], prompt, is_test=False)
print(response)

Elon Musk's funniest tweet was posted on March 31, 2018: "It’s time to investigate the investigation."


In [84]:
prompt = "Are Elon Musk's internet presence worth the hype? Is he worth following on Twitter and why?"
response = get_summaries_from_transcript(chunk[0:100], prompt, is_test=False)
print(response)

Elon Musk's internet presence is definitely worth the hype, and yes, it is worth following him on Twitter. He often posts news about new developments in his companies, such as SpaceX, Tesla, and The Boring Company. He also shares inspiring stories about his personal life and insights into his thought process, which are interesting for both enthusiasts and casual observers. On the whole, following Elon Musk on Twitter is a great way to stay informed and get inspired.


In [85]:
prompt = "Based on Elon Musk's previous tweets, generate his next tweet."
response = get_summaries_from_transcript(chunk[0:100], prompt, is_test=False)
print(response)

Excited for our new Tesla Superchargers #across the globe and giving the world even greater #access to sustainable #transportation! #EV
